# Reading MODIS data with xarray

## Import python packages

In [ ]:
import pandas as pd
import s3fs
import xarray as xr

## Connect to bucket (anonymous login for public data only)

In [ ]:
fs = s3fs.S3FileSystem(
    anon=True, client_kwargs={"endpoint_url": "https://climate.uiogeo-apps.sigma2.no/"}
)

In [ ]:
fs.ls("ESGF/obs4MIPs/MODIS/MODIS6.1terra")[:10]

In [ ]:
s3path = "ESGF/obs4MIPs/MODIS/MODIS6.1terra/*od550aer*.nc"
remote_files = fs.glob(s3path)
print(remote_files)

## Access data files

In [ ]:
# Iterate through remote_files to create a fileset
fileset = [fs.open(file) for file in remote_files]

## Data reading with xarray

In [ ]:
# Read file with xarray
dset = xr.open_mfdataset(fileset, combine="by_coords")

In [ ]:
dset

## Plot a single time

In [ ]:
!pip install cmaps

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cmaps

In [ ]:
fig = plt.figure(figsize=(20, 10))

# We're using cartopy and are plotting in Orthographic projection
# (see documentation on cartopy)
ax = plt.subplot(
    1, 1, 1, projection=ccrs.PlateCarree(central_longitude=20.0, globe=None)
)
ax.coastlines(resolution="110m")

# custom colormap
lcmap = cmaps.BlueYellowRed

# We need to project our data to the new Mercator projection and for this we use `transform`.
# we set the original data projection in transform (here PlateCarree)
# we only plot values greather than 0
dset["od550aer"].sel(time="2001-01-01").plot(
    ax=ax, transform=ccrs.PlateCarree(), cmap=lcmap, vmin=0.0, vmax=0.5
)
ax.set_title("MODIS - 2001-01-01\n ", fontsize=20)

## Interactive plot

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from datetime import datetime

In [ ]:
def plot_map(date):
    fig = plt.figure(figsize=(20, 10))

    ax = plt.subplot(
        1, 1, 1, projection=ccrs.PlateCarree(central_longitude=20.0, globe=None)
    )
    ax.coastlines(resolution="110m")

    # custom colormap
    lcmap = cmaps.BlueYellowRed

    # We need to project our data to the new Mercator projection and for this we use `transform`.
    # we set the original data projection in transform (here PlateCarree)
    # we only plot values greather than 0
    dset["od550aer"].sel(time=date).plot(
        ax=ax, transform=ccrs.PlateCarree(), cmap=lcmap, vmin=0.0, vmax=0.5
    )
    ax.set_title("MODIS - {}\n ".format(date), fontsize=20)

In [ ]:
start_date = datetime(2000, 1, 1)
end_date = datetime(2018, 12, 31)

dates = pd.date_range(start_date, end_date, freq="D")

options = [(date.strftime("%Y-%m-%d"), date) for date in dates]
index = (0, len(options) - 1)

date_slider = widgets.SelectionSlider(
    options=options, orientation="horizontal", layout={"width": "800px"}
)

In [ ]:
interact(plot_map, date=date_slider);